In [1]:
import pandas as pd
import numpy as np

In [2]:
# import dataset into a pandas data frame
df = pd.read_csv('41467_2022_32829_MOESM5_ESM.csv')

df.head()

,ID,DNA1 [counts],DNA2 [counts],DNA3 [counts],RNA1 [counts],RNA2 [counts],RNA3 [counts],TX1 [au],TX2 [au],TX3 [au],...,high quality,Observed log(TX/Txref),Predicted log(TX/Txref),dG10,dG35,dGDisc,dGITR,dGEXT10,dGSPAC,dGUP
0,0,8263,7261,5173,16341,10320,13506,2.258071,1.523795,1.545541,...,Yes,-3.386326,-3.844827,-1.781524,-1.477218,-0.106428,-0.021112,0.191352,-0.0924,0.400862
1,1,5600,4886,3264,10986,7250,10800,2.240001,1.590845,1.958709,...,Yes,-3.503140,-3.905283,-1.781524,-1.477218,-0.166884,-0.021112,0.191352,-0.0924,0.400862
2,2,7898,6790,4752,19572,32204,30585,2.829533,5.084911,3.810029,...,Yes,-4.207206,-3.905283,-1.781524,-1.477218,-0.166884,-0.021112,0.191352,-0.0924,0.400862
3,3,10651,9875,6466,15734,16246,18908,1.686729,1.763814,1.731036,...,Yes,-3.392439,-3.877808,-1.781524,-1.477218,-0.139409,-0.021112,0.191352,-0.0924,0.400862
4,4,12188,10793,6965,28609,21796,26803,2.680198,2.165100,2.278025,...,Yes,-3.698903,-3.672384,-1.781524,-1.477218,0.066015,-0.021112,0.191352,-0.0924,0.400862


In [3]:
# Function to one-hot encode DNA sequences

def one_hot_encode(sequence):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    encoding = np.zeros((len(sequence), len(mapping)))
    for i, nucleotide in enumerate(sequence):
         encoding[i, mapping[nucleotide]] = 1
    return encoding

In [4]:
# one-hot encode the DNA sequence

X = np.array([one_hot_encode(h35 + h10) for h35, h10 in zip(df['h35'], df['h10'])])
y = df['Observed log(TX/Txref)']

X.shape

(13481, 12, 4)

In [5]:
from sklearn.model_selection import train_test_split

# split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)

In [18]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define RNN model architecture
model = Sequential()
model.add(LSTM(32, input_shape=(12, 4)))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test Loss (mse):", loss)


Epoch 1/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.4735 - val_loss: 0.4014
Epoch 2/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4057 - val_loss: 0.3800
Epoch 3/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3692 - val_loss: 0.3006
Epoch 4/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3034 - val_loss: 0.2314
Epoch 5/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2326 - val_loss: 0.2104
Epoch 6/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2281 - val_loss: 0.2081
Epoch 7/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2179 - val_loss: 0.2131
Epoch 8/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2313 - val_loss: 0.2072
Epoch 9/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2225 - val_loss: 0.2045
Epoch 10/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2200 - val_loss: 0.2040
Epoch 11/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2149 - val_loss: 0.2092
Epoch 12/100
169/169 ━━━━━━━━━━━━━━━━━━━━

In [19]:
# save the model
model.save('my_model.keras')